In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import pickle
import os

In [43]:
import keras
keras.__version__
from tensorflow.keras.utils import normalize
import numpy as np
from matplotlib import pyplot as plt
import glob
import cv2
import os
from keras import backend as K
import tensorflow as tf

In [2]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [3]:
import keras
keras.__version__

'2.5.0'

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Flatten, MaxPooling2D

In [5]:
from datetime import datetime
from sklearn import metrics

In [6]:
SIZE = 512
DIR = "mini_brain/"
print(os.listdir("mini_brain/"))

['all_images', 'all_labels', 'test_images', 'test_labels']


In [7]:
train_images = []

for directory_path in glob.glob(DIR + "all_images"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)

In [8]:
train_masks = []

for directory_path in glob.glob(DIR + "all_labels"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (SIZE, SIZE), interpolation = cv2.INTER_NEAREST)
        train_masks.append(mask)

In [74]:
train_images = np.array(train_images)
train_masks = np.array(train_masks)

In [75]:
np.unique(train_masks)

array([  0, 128, 255], dtype=uint8)

In [76]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1, 1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

C:\Users\alif-\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [77]:
train_masks = train_masks_encoded_original_shape

In [78]:
np.unique(train_masks)

array([0, 1, 2], dtype=int64)

In [79]:
print(train_images.shape)
print(train_masks.shape)

(16, 512, 512, 3)
(16, 512, 512)


In [80]:
X_train = train_images
y_train = train_masks
y_train = np.expand_dims(y_train, axis=3)

In [81]:
print(X_train.shape)
print(y_train.shape)

(16, 512, 512, 3)
(16, 512, 512, 1)


### Conv Layers

In [14]:
activation = 'sigmoid'
feature_extractor = Sequential()
feature_extractor.add(Conv2D(32, 3, activation=activation, 
        padding='same', input_shape=(SIZE, SIZE, 3)))
feature_extractor.add(Conv2D(32, 3, activation=activation, 
        padding='same', kernel_initializer='he_uniform'))
# feature_extractor.add(Conv2D(64, 3, activation=activation, 
#         padding='same', kernel_initializer='he_uniform'))
# feature_extractor.add(BatchNormalization())
# feature_extractor.add(Conv2D(64, 3, activation=activation, 
#         padding='same', kernel_initializer='he_uniform'))
# feature_extractor.add(BatchNormalization())
# feature_extractor.add(MaxPooling2D())
# feature_extractor.add(Flatten())

In [83]:
X = feature_extractor.predict(X_train)

In [55]:
# X = np.moveaxis(X, 0, 1)

In [84]:
X = X.reshape(-1, X.shape[3])
Y = y_train.reshape(-1)

In [85]:
print(X.shape)
print(Y.shape)

(4194304, 32)
(4194304,)


In [86]:
dataset = pd.DataFrame(X)
dataset['Label'] = Y
print(dataset['Label'].unique())

[0 2 1]


In [87]:
print(dataset['Label'].value_counts())

0    4146224
2      30022
1      18058
Name: Label, dtype: int64


In [60]:
# dataset = dataset[dataset['Label'] != 0]

In [89]:
X_for_RF = dataset.drop(labels = ['Label'], axis=1)
Y_for_RF = dataset['Label']

### Random Forest

In [63]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [64]:
%%time
model.fit(X_for_RF, Y_for_RF)

Wall time: 41min 17s


RandomForestClassifier(n_estimators=50, random_state=42)

In [9]:
filename = 'rf_conv_50est_49img.sav'

In [93]:
pickle.dump(model, open(filename, 'wb'))

### Predict

In [10]:
loaded_model = pickle.load(open(filename, 'rb'))

In [68]:
test_fname = 'P044'
test_format = '.png'

In [69]:
test_img_path = DIR + 'test_images/' + test_fname + test_format
test_img = cv2.imread(test_img_path, cv2.IMREAD_COLOR)
test_img = cv2.resize(test_img, (SIZE, SIZE))
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
test_img = np.expand_dims(test_img, axis=0)

In [70]:
X_test_features = feature_extractor.predict(test_img)
X_test_features = X_test_features.reshape(-1, X_test_features.shape[3])

In [71]:
X_test_features.shape

(262144, 32)

In [72]:
prediction = loaded_model.predict(X_test_features)

In [73]:
# Reshape and save prediction
prediction_image = prediction.reshape(mask.shape)
plt.imsave("rf_multi_conv/predicted/" + test_fname + test_format, prediction_image, cmap="binary")

In [74]:
true_mask_path = DIR + 'test_labels/' + test_fname + test_format
true_mask = cv2.imread(mask_path, 0)
true_mask = cv2.resize(mask, (SIZE, SIZE), interpolation = cv2.INTER_NEAREST)

In [76]:
true_mask.shape

(512, 512)

In [78]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
h, w = true_mask.shape
true_mask_reshaped = true_mask.reshape(-1, 1)
true_mask_reshaped_encoded = labelencoder.fit_transform(true_mask_reshaped)
true_mask_encoded_original_shape = true_mask_reshaped_encoded.reshape(h, w)

In [79]:
true_mask = true_mask_reshaped_encoded

In [80]:
np.unique(true_mask)

array([0, 1, 2], dtype=int64)

In [81]:
true_mask = true_mask.reshape(-1)

In [82]:
print(true_mask.shape)
print(prediction.shape)

(262144,)
(262144,)


In [83]:
print(metrics.accuracy_score(true_mask, prediction))

0.9827651977539062


### Jaccard

In [84]:
# IOU for each class is the following
# IOU = true_pos / (true_pos + false_pos + false_neg)

In [85]:
jaccard_score = metrics.jaccard_score(true_mask, prediction, average='macro')
print(jaccard_score)

0.32758839925130206


### IOU

In [86]:
tf.keras.metrics.MeanIoU

tensorflow.python.keras.metrics.MeanIoU

In [87]:
print(true_mask.shape)
print(prediction.shape)

(262144,)
(262144,)


In [88]:
print(np.unique(true_mask))
print(np.unique(prediction))

[0 1 2]
[0]


In [89]:
# from keras.metrics import MeanIOU
num_classes = 3
IOU_RF = tf.keras.metrics.MeanIoU(num_classes=num_classes)
IOU_RF.update_state(true_mask, prediction)
print(IOU_RF.result().numpy())

0.3275884


In [90]:
values = np.array(IOU_RF.get_weights()).reshape(num_classes, num_classes)
print(values)

[[257626.      0.      0.]
 [  1390.      0.      0.]
 [  3128.      0.      0.]]


In [91]:
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2]  + values[1,0]+ values[2,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2]  + values[0,1]+ values[2,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[0,2]+ values[1,2])

In [92]:
print("IoU for class 1 is: ", class1_IoU)
print("IoU for class 2 is: ", class2_IoU)
print("IoU for class 3 is: ", class3_IoU)

IoU for class 1 is:  0.9827652
IoU for class 2 is:  0.0
IoU for class 3 is:  0.0
